In [3]:
from __future__ import absolute_import, division, print_function
from __future__ import absolute_import, division, print_function

import sys
import os

import pandas as pd
import numpy as np
from scipy.stats import norm
import itertools


# TSA from Statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.5f' % x) # pandas
np.set_printoptions(precision=5, suppress=True) # numpy

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# seaborn plotting style
sns.set(style='ticks', context='poster')

#############################################


df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_15min_no_outliers.csv', index_col=0, parse_dates=[0])

df.power = df.power.interpolate(method='linear', limit = 1 )
df.speed = df.speed.interpolate(method='linear', limit = 1 )

ts_df = df
n_sample = ts_df.shape[0] 
    

# Create a training sample and testing sample before analyzing the series
n_train=int(0.95*n_sample)+1
n_forecast=n_sample-n_train

ts_train = ts_df.iloc[:n_train]['power']
ts_test = ts_df.iloc[n_train:]['power']
ts_train_speed = ts_df.iloc[:n_train]['speed']
ts_test_speed = ts_df.iloc[n_train:]['speed']


# iputs for model and prediction
endog = ts_df.ix[:, 'power']
exog = sm.add_constant(ts_df.ix[:, 'speed'])
nobs = endog.shape[0]


In [4]:
#Model Estimation

# grid parameters to test
p_min = 0
d_min = 0
q_min = 0
p_max = 5
d_max = 0
q_max = 5

# Initialize a DataFrame to store the results
results_bic = pd.DataFrame(index=['AR{}'.format(i) for i in range(p_min,p_max+1)],
                           columns=['MA{}'.format(i) for i in range(q_min,q_max+1)])

for p,d,q in itertools.product(range(p_min,p_max+1),
                               range(d_min,d_max+1),
                               range(q_min,q_max+1)):
    if p==0 and d==0 and q==0:
        results_bic.loc['AR{}'.format(p), 'MA{}'.format(q)] = np.nan
        continue

    try:
        model = sm.tsa.SARIMAX(ts_train, order=(p, d, q)  )
        results = model.fit()
        results_bic.loc['AR{}'.format(p), 'MA{}'.format(q)] = results.bic
    except:
        continue
results_bic = results_bic[results_bic.columns].astype(float)


fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(results_bic,
                 mask=results_bic.isnull(),
                 ax=ax,
                 annot=True,
                 fmt='.2f',
                 )
ax.set_title('BIC')

plt.show()


# FOUND PARAMS
# 1h params: ar(p) 2, ma(q) 2  OR ar(p) 3, ma(q) 1
# 30min params: ar(p) 2, ma(q) 5
# 15min params: ar(p) 2, ma(q) 2  


C:\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\tools.py:405: RuntimeWarning: invalid value encountered in sqrt
  x = r / ((1 - r**2)**0.5)
C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:475: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\statsmodels\tools\numdiff.py:96: RuntimeWarning: invalid value encountered in maximum
  h = EPS**(1. / s) * np.maximum(np.abs(x), 0.1)
C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:475: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:475: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:475: ConvergenceWarning: Maximum Likelihoo